In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
import plotly
import math

In [2]:
channels=pd.read_excel("Preprocessed_EEG/channel-order(viz).xlsx")
channelNames=channels.iloc[:,0]
channelNames=np.ndarray.tolist(pd.Series.to_numpy(channelNames))
channelNames.insert(0,"Fp1")
info=mne.create_info(channelNames,200,'eeg')
subject=loadmat('1_20131107.mat')
ch_types = ['eeg'] * len(channelNames)
info = mne.create_info(channelNames, ch_types=ch_types, sfreq=200)
info.set_montage('standard_1020')
data=np.delete(subject['djc_eeg1'], 61, 0)
data=np.delete(data, 57, 0)
evoked=mne.EvokedArray(data,info)
transposedData=np.transpose(data)
df = pd.DataFrame(transposedData, columns = channelNames)
df = df.head(500)

In [4]:
def makeValueMatrix(currentAnalysed):
    result=pd.read_excel("Preprocessed_EEG/channel-order(topo).xlsx").to_numpy().astype(np.float32)
    #tracker value
    k=0
    for i in range(0,result.shape[0]):
        for j in range (0,result.shape[1]):
            if not(math.isnan(result[i][j])):
                result[i][j]=currentAnalysed[k]
                k+=1
    
    return result

In [5]:
def makePostionList(data):
    position=[]
    for column in range (0,500):
        position.append(makeValueMatrix(data[:,column]))
    return position

In [6]:
points=makePostionList(data)

In [59]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
app = JupyterDash(__name__)
slider = html.Div(
    [
        html.Label("Select Time Frame", htmlFor="time"),
        dcc.Slider(
            min=0,
            max=500,
            step=5,
            value=0,
            id="time",
            marks=None,
            tooltip={"placement": "bottom", "always_visible": True},
        ),
    ]
)
app.layout = html.Div(className='row', children=[
                 slider,
                 #container for linechart and heatMap
                 html.Div(
                     children=[
                     #heatmap div
                     html.Div(
                        children=[
                            dcc.Graph(id="heatmap"),
                     ],style={'display': 'inline-block'}),
                     #line chart div
                     html.Div(
                         children=[
                            dcc.Graph(id="lineChart")
                     ],style={'display': 'inline-block'})
                 ])
                ])
@app.callback(Output("heatmap", "figure"),
              Output("lineChart", "figure"),
              Input("time", "value"))
def updateFigures(time_instance):
    heatMap = px.imshow(points[time_instance])
    heatMap.update_xaxes(showticklabels=False)
    heatMap.update_yaxes(showticklabels=False)
    heatMap.update_layout(xaxis=dict(scaleanchor='y', constrain='domain'),
                  width=550,
                  height=500 
                 )
    lineChart= px.line(df.head(time_instance))
    return heatMap, lineChart
if __name__ == "__main__":
    app.run_server()

Dash app running on http://127.0.0.1:8050/
